# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.hyperdrive import MedianStoppingPolicy
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.webservice import AciWebservice

import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

In [ ]:
data_source = "https://github.com/Graflinger/Azure-ML-Engineer-Udacity-3/blob/392ffdc670986f86205b84fbc9cb57a82c818389/water_potability.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_source)

df = ds.to_pandas_dataframe()
df = df.apply(lambda x: x.fillna(x.mean()),axis=0)
df.to_csv("prepared_data.csv")

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

Answer: The model used is a RandomForest Classifier. It is an Essemble of many desicion trees. The 3 picked hyperparameters are chosen, because these are the most common ones regarding a Random Forest. The n_estimator defines, how many desicion trees are being used. The criterion is about the function to measure the quality of a split and the max depth defines the max depth of any given tree in the forest. For termination poilcy, the bandit Policy was chosen to terinate runs, which will not provide a good result to save performance. In the config settings, the primary metric is defined (Accuracy) and that this should be maximized. Furthermore its configurated, that 20 runs of this hyperdrive should get excecuted, 4 at the same time.

In [ ]:
myenv = Environment.from_conda_specification(name="myenv", file_path = './dependencies.yml')

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
  "--in_n_estimator": choice(16,32,64,128),
  "--in_criterion": choice("gini", "entropy"),
  "--in_max_depth": choice(2,4,6,8,16)
})

# set up script run configuration
config = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=my_compute_target,
    environment=myenv,
)

config.run_config.target = my_compute_target

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=config, 
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=early_termination_policy, 
    max_total_runs=20,
    max_concurrent_runs=4
    )

In [ ]:
# Submit the hyperdrive run to the experiment and show run details with the widget.
exp = Experiment(workspace=ws, name=experiment_name)
hyperdrive_submit = exp.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_submit).show()
hyperdrive_submit.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get the best run and save the model from that run.

best_run = hyperdrive_submit.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


In [ ]:
#TODO: Save the best model
joblib.dump(best_run, 'best_model_hyperdrive.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_run.register_model(model_name='hyperdrive_best', model_path='outputs/model_forest.joblib')

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=4, auth_enabled=True, enable_app_insights=True
)

service = Model.deploy(workspace=ws,
                          name="deployment",
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=deployment_config,
                          overwrite=True)
service.wait_for_deployment(show_output=True)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "data":
        [   [3.71608007538699,129.42292051494425,18630.057857970347,6.635245883862,8.05933237743854,92.8853591348523,398.41081338184466,11.558279443446395,31.997992727424737,4.075075425430034]]
}

body = str.encode(json.dumps(data))

url = 'http://c55be43a-d2ba-4418-9e9f-3f3f8c71f99e.southcentralus.azurecontainer.io/score'
api_key = '8y8WxM75DEsAdy6lo0utyhEfMpiCiWCa' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Deleting service
service.delete()